## Home Work Three - 2025 ML Zoomcamp 

## By: Norman Angel

## Date: Oct 13, 2025

## Task

- In this homework, we will use the lead scoring dataset Bank Marketing dataset

- In this dataset our desired target for classification task will be converted variable - has the client signed up to the platform or not.

In [1]:
# Importing Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Load the dataset 

df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


## Data preparation

- Check if the missing values are presented in the features.
- If there are missing values:
- For caterogiral features, replace them with 'NA'
- For numerical features, replace with with 0.0


In [3]:
# Check datatypes

df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [5]:
# Check Missing values

df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [7]:
# Fill categorical missing values  in df with 'NA'

categorical = df.select_dtypes(include=['object', 'category']).columns
df[categorical] = df[categorical].fillna('NA')

In [8]:
# and numerical missing values with 0.0

numerical = df.select_dtypes(include=['int64', 'float64']).columns
df[numerical] = df[numerical].fillna(0.0)

In [9]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64